import libraries

In [4]:
import tmdbsimple
import requests
import pandas as pd
import time

Loop through 50 pages to create a dataframe of 1000 movies

In [11]:
key = open('key.txt','r').read()
payload = "{}"

movie_df = pd.DataFrame() #create empty dataframe to enable 'while loop' below

page=1
while movie_df.shape[0] < 1000:
    url = "https://api.themoviedb.org/3/movie/top_rated?api_key={0}&language=en-US&page={1}".format(key, str(page))
    response = requests.request("GET", url, data=payload).json()
    if page == 1: #initialize dataframe on first loop
        movie_df = pd.DataFrame(response['results'])
    else:
        movie_df = movie_df.append(pd.DataFrame(response['results']))
    
    movie_df = movie_df[movie_df['original_language']=='en'] #remove non english movies
    time.sleep(0.25) #rate limit is 4 pages per second
    page+=1
        

In [13]:
movie_df.to_csv('data\movie_df.csv')
movie_df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
1,False,/xBKGJQsAIeweesB79KC89FpBrVr.jpg,"[18, 80]",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,28.527767,/9O7gLzmreU0nGkIB6K3BsJbzvNv.jpg,1994-09-23,The Shawshank Redemption,False,8.5,9773
3,False,/6xKCYgH16UuwEGAyroLU6p8HLIn.jpg,"[18, 80]",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",36.965452,/rPdtLWNsZmAtoZl9PK7S2wE3qiS.jpg,1972-03-14,The Godfather,False,8.5,7394
5,False,/rIpSszng8P0DL0TimSzZbpfnvh1.jpg,"[18, 36, 10752]",424,en,Schindler's List,The true story of how businessman Oskar Schind...,19.945455,/yPisjyLweCl1tbgwgtzBCNCBle.jpg,1993-11-29,Schindler's List,False,8.4,5518
6,False,/gLbBRyS7MBrmVUNce91Hmx9vzqI.jpg,"[18, 80]",240,en,The Godfather: Part II,In the continuing saga of the Corleone crime f...,30.191804,/tHbMIIF51rguMNSastqoQwR0sBs.jpg,1974-12-20,The Godfather: Part II,False,8.4,4249
7,False,/xqjGKLwLZeujg4fiBTOqhZkoL31.jpg,"[18, 9648]",452522,en,Twin Peaks,Standalone version of the series pilot with an...,5.969249,/zQsEi6096L7PvowV39dtdqdW16f.jpg,1989-12-31,Twin Peaks,False,8.4,123


In [4]:
movie_df.shape

(1000, 14)

In [6]:
time.sleep(0.25)

In [7]:
movie_df = pd.DataFrame()

In [9]:
movie_df.shape

(0, 0)